## Data Prep

In [4]:
# load in the dependancies
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import random

In [7]:
# image processing function
def create_training_data():
    
    # Use binary classes as labels of the directory
    for category in Classes:

        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)

        # loop through each labeled directoryu
        for img in os.listdir(path):

            # Add images to training set
            try:
                img_path = os.path.join(path, img)
                img_array = cv2.imread(img_path)

                # normalize the image for optimal parsing
                img_size = 224
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_Data.append([new_array, class_num])

            except Exception as e:
                # print('Failed to Grab Image')
                pass

In [8]:
# basic variables
img_size = 224
Datadirectory = 'Dataset/'
Classes = ['Louis', 'Not_Louis'] # i'm using class labels as the local directory names as-well

# training set
training_Data = []
create_training_data()
random.shuffle(training_Data)

X = [] # Features
y = [] # label

for features, label in training_Data:
    X.append(features)
    y.append(label)

# add image features to numpy matrix for max efficiency
X = np.array(X).reshape(-1, img_size, img_size, 3)
Y = np.array(y)
X = X/255

Failed to Grab Image
Failed to Grab Image


## Training

In [19]:
# load-in pre-trained keras model (lightweight image processing DL structure)
model = tf.keras.applications.mobilenet.MobileNet()

# extra junk needed to normalize the final layers of the model
base_input = model.layers[0].input
base_ouput = model.layers[-4].output
Flat_layer = layers.Flatten()(base_ouput)
final_output = layers.Dense(1)(Flat_layer)

# define binary classification nature
final_output = layers.Activation('sigmoid')(final_output)
new_model = keras.Model(inputs = base_input, outputs = final_output)
new_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# fit the bad boy
new_model.fit(X, Y, epochs = 15, validation_split = 0.2)

Epoch 1/15
9/9 [==============================] - 27s 3s/step - loss: 0.3024 - accuracy: 0.8490 - val_loss: 1.0670 - val_accuracy: 0.8310
Epoch 2/15
9/9 [==============================] - 21s 2s/step - loss: 0.2371 - accuracy: 0.9596 - val_loss: 2.5358 - val_accuracy: 0.6901
Epoch 3/15
9/9 [==============================] - 22s 2s/step - loss: 0.0292 - accuracy: 0.9872 - val_loss: 5.0570 - val_accuracy: 0.5915
Epoch 4/15
9/9 [==============================] - 22s 2s/step - loss: 0.0347 - accuracy: 0.9890 - val_loss: 11.1590 - val_accuracy: 0.5070
Epoch 5/15
9/9 [==============================] - 22s 2s/step - loss: 7.0599e-04 - accuracy: 1.0000 - val_loss: 9.2672 - val_accuracy: 0.5211
Epoch 6/15
9/9 [==============================] - 21s 2s/step - loss: 0.0907 - accuracy: 0.9769 - val_loss: 3.3865 - val_accuracy: 0.7324
Epoch 7/15
9/9 [==============================] - 21s 2s/step - loss: 0.0776 - accuracy: 0.9879 - val_loss: 6.1345 - val_accuracy: 0.6620
Epoch 8/15
9/9 [=============

In [20]:
# save the model
new_model.save('./Model/Fitted_Model.h5')